In [1]:
import caveclient
import pandas as pd
import numpy as np
import neuroglancer
import neuroglancer.static_file_server
import json


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
client = caveclient.CAVEclient('minnie65_phase3_v1')


In [3]:
ct_df=client.materialize.query_table('aibs_soma_nuc_metamodel_preds_v117')

In [4]:
# spec of a segment info file here
# https://github.com/google/neuroglancer/blob/master/src/datasource/precomputed/segment_properties.md

def make_tag_entry(taglists, tagdescription_ds, description="tags"):
    unique_tag_lists = [np.unique(tl).tolist() for tl in taglists]
    
    # concatenate all unique tag lists
    unique_tags = np.concatenate(unique_tag_lists).tolist()

    # for each unique_tag_list get the description
    unique_tag_descs = [[tag_d[t] for t in tl] for tl, tag_d in zip(unique_tag_lists, tagdescription_ds)]

    # concatenate all unique tag descriptions
    unique_tag_desc = np.concatenate(unique_tag_descs).tolist()

    # for each tag list, get the indices of the unique tags
    tag_indices = [[unique_tags.index(t) for t in tl] for tl in taglists]
    tag_indices = [list(tags) for tags in zip(*tag_indices)]
    return {"id": "tags",
            "type": "tags",
            "description": description,
            "tags": unique_tags,
            "tag_descriptions": unique_tag_desc,
            "values": tag_indices
           }

unique_ct = ct_df.cell_type.unique()
unique_class = ct_df.classification_system.unique()

ct_desc = {c:c for c in unique_ct}
ct_desc['PTC'] = 'Proximal targetting interneuron'
ct_desc['DTC'] = "Distal targetting interneuron"
ct_desc['ITC'] = "Interneuron targetting interneuron"
ct_desc['STC'] = "Sparsely targetting interneuron"
ct_desc['L5NP']= "Layer 5 Near Projecting"

cell_class_desc = {c:c for c in unique_class}


tag_entry=make_tag_entry([ct_df.cell_type, ct_df.classification_system],
                          [ct_desc, cell_class_desc])

segment_info ={"@type":  "neuroglancer_segment_properties",
        "inline": {
            "ids": [str(i) for i in ct_df.pt_root_id.values.tolist()],
            "properties": [
                {"id": "label",
                 "type": "label",
                 "description": "filename",
                 "values": [str(v) for v in ct_df.pt_root_id.values]},
                tag_entry,
            ]
        }
}

In [5]:
# write the segment info to a file
segment_info_file = "info"
with open(segment_info_file, "w") as f:
    json.dump(segment_info, f)
    

In [6]:
server = neuroglancer.static_file_server.StaticFileServer(
        static_dir='.', bind_address= "127.0.0.1", daemon=True
    )

In [7]:
# this is what you want to enter into neuroglancer
# hit plus under the source of the segmentation layer you want
# to add these segment properties to and paste in 
'precomputed://'+server.url

'precomputed://http://127.0.0.1:59221/7695f672e1ea946d4377593b7b814844183c0e93'